# DocuRAG — Document Retrieval‑Augmented Generation (RAG)

**DocuRAG** is a production‑style, modular Retrieval‑Augmented Generation (RAG) system designed to demonstrate how modern LLM applications retrieve, ground, and generate answers over user‑supplied documents — without LangChain.

It is built to showcase applied ML, NLP, and systems design skills relevant to real‑world RAG deployments.

## Modularity

Each step in the RAG pipeline is isolated:

### Ingestion → Extraction → Chunking → Embedding → Retrieval → Generation, followed by UI + reset

- `ingestion.py` handles upload/URL

- `extraction.py` does per-page cascade + optional OCR

- `chunking.py` supports dual chunking + auto

- `vectorstore.py` ensures unique temp Chroma dirs

- `rag.py` orchestrates indexing + retrieval + summary intent + generation

- `ui/gradio_app.py` is a thin UI layer (good practice)

DocuRAG enables users to ask natural‑language questions over custom knowledge sources provided via:

- 📂 Local PDF upload
- 🌐 URL ingestion (automatic fetch + processing)
  
The system retrieves the most relevant document segments using vector similarity search (Chroma) and generates context‑grounded answers using an LLM, with optional summary‑style responses when intent is detected.

**Key design goals**:

- Clear separation of concerns
- Extensibility across models and embeddings
- Robust document ingestion (including OCR)
- Session‑isolated retrieval to avoid data leakage

## Install dependencies

In [2]:
!pip install -q openai chromadb pymupdf gradio python-dotenv requests nltk

## Imports

In [3]:
import os
import re
import uuid
import tempfile
import shutil
import requests
from typing import List, Dict, Tuple, Optional

import fitz  # PyMuPDF
import nltk
import gradio as gr
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from openai import OpenAI

# Load .env / environment variables (Codespaces + HF Spaces compatible)
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

openai_client = OpenAI(api_key=api_key)

print("✅ OpenAI client configured successfully.")
print(api_key[:15], "********")


✅ OpenAI client configured successfully.
sk-proj-6-J1Chx ********


## Optional OCR dependency (safe import)

In [4]:
try:
    import pdfplumber
    PDFPLUMBER_AVAILABLE = True
except ImportError:
    PDFPLUMBER_AVAILABLE = False


## Sentence tokenizer setup

In [5]:
def ensure_nltk_resources():
    """
    Ensures sentence tokenizer resources exist.
    Avoids punkt/punkt_tab runtime crashes in Codespaces/HF.
    """
    for res in ["punkt", "punkt_tab"]:
        try:
            nltk.data.find(f"tokenizers/{res}")
        except LookupError:
            nltk.download(res)

ensure_nltk_resources()
from nltk.tokenize import sent_tokenize

def safe_sentence_split(text: str) -> List[str]:
    """
    Sentence splitter with regex fallback if NLTK resources fail.
    """
    try:
        return sent_tokenize(text)
    except Exception:
        return re.split(r'(?<=[.!?])\s+', text)


# INGESTION

#### Ingestion: Upload OR URL → local PDF path

In [6]:
def ingest_pdf(file=None, url: Optional[str] = None) -> Tuple[str, str]:
    """
    Ingestion stage:
    - If a file is uploaded, return its path.
    - If a URL is provided, download to /tmp and return local path.
    
    Returns:
      (local_path, source_name)
    """
    if file is not None:
        if not file.name.lower().endswith(".pdf"):
            raise ValueError("Uploaded file is not a PDF.")
        return file.name, os.path.basename(file.name)

    if url and url.strip():
        try:
            resp = requests.get(url.strip(), timeout=30)
            resp.raise_for_status()
            local_path = f"/tmp/{uuid.uuid4().hex}.pdf"
            with open(local_path, "wb") as f:
                f.write(resp.content)

            source_name = url.strip().split("/")[-1] or "downloaded.pdf"
            if not source_name.lower().endswith(".pdf"):
                source_name += ".pdf"

            return local_path, source_name
        except Exception as e:
            raise ValueError(f"Failed to download PDF: {e}")

    raise ValueError("Please upload a PDF or provide a PDF URL.")

# EXTRACTION
#### Text cleaning (improves extraction + embeddings)

In [8]:
def clean_text(text: str) -> str:
    """
    Cleaning stage:
    - Removes hyphen line breaks
    - Flattens newlines
    - Collapses repeated whitespace
    """
    if not text:
        return ""
    text = text.replace("-\n", "")
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# Optional TRUE OCR dependencies (pytesseract requires system 'tesseract-ocr' binary)
try:
    import pytesseract
    from PIL import Image
    PYTESSERACT_AVAILABLE = True
except ImportError:
    PYTESSERACT_AVAILABLE = False

def ocr_page_with_tesseract(page: fitz.Page, dpi: int = 200) -> str:
    """Render a PDF page to an image and run TRUE OCR with Tesseract (if available).

    Notes:
    - This is only used as a *last resort* when text extraction yields nothing.
    - Requires `pytesseract` (python) + `tesseract-ocr` (system package).
    """
    if not PYTESSERACT_AVAILABLE:
        return ""
    try:
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        return clean_text(pytesseract.image_to_string(img))
    except Exception:
        # Keep OCR failure non-fatal; downstream will handle empty extraction.
        return ""


#### Extraction: PDF → per-page text (PyMuPDF, optional OCR fallback)

In [9]:
def extract_pages_with_cascade(file_path: str) -> Tuple[List[str], Dict]:
    """Extraction stage with per-page extractor cascade.

    Cascade per page:
      1) PyMuPDF (fitz) text extraction
      2) pdfplumber text-layer extraction (if installed)
      3) TRUE OCR via pytesseract (if available)

    Returns:
      pages_text: list[str] cleaned text per page (may contain empty strings)
      stats: dict with extraction counts (useful for status/debug)
    """
    pages_text: List[str] = []
    stats = {
        "pages_total": 0,
        "fitz_pages_with_text": 0,
        "plumber_pages_with_text": 0,
        "ocr_pages_with_text": 0,
        "empty_pages": 0,
        "pytesseract_available": PYTESSERACT_AVAILABLE,
        "pdfplumber_available": PDFPLUMBER_AVAILABLE,
    }

    doc = fitz.open(file_path)
    stats["pages_total"] = len(doc)

    plumber_pdf = None
    if PDFPLUMBER_AVAILABLE:
        try:
            import pdfplumber  # local import to keep optional dependency truly optional
            plumber_pdf = pdfplumber.open(file_path)
        except Exception:
            plumber_pdf = None

    for page_num in range(len(doc)):
        page = doc[page_num]

        # 1) Primary: fitz
        txt = clean_text(page.get_text("text") or "")
        if txt.strip():
            stats["fitz_pages_with_text"] += 1
            pages_text.append(txt)
            continue

        # 2) Secondary: pdfplumber (text layer)
        if plumber_pdf is not None:
            try:
                ptxt = clean_text(plumber_pdf.pages[page_num].extract_text() or "")
            except Exception:
                ptxt = ""
            if ptxt.strip():
                stats["plumber_pages_with_text"] += 1
                pages_text.append(ptxt)
                continue

        # 3) Last resort: TRUE OCR
        otxt = ocr_page_with_tesseract(page) if PYTESSERACT_AVAILABLE else ""
        if otxt.strip():
            stats["ocr_pages_with_text"] += 1
            pages_text.append(otxt)
        else:
            stats["empty_pages"] += 1
            pages_text.append("")

    if plumber_pdf is not None:
        try:
            plumber_pdf.close()
        except Exception:
            pass
    doc.close()

    return pages_text, stats


# CHUNKING

#### Smaller, safer chunking with overlap (returns metadata)

In [10]:
# ✅ Drop-in chunking implementations (LangChain-free)
# We expose 3 strategies:
# 1) Word-window (robust for long/noisy/OCR text)
# 2) Sentence-based (more readable citations; great for summaries)
# 3) Unified controller ("auto") that chooses based on page length

def chunk_word_window(
    text: str,
    source_name: str,
    page_num: int,
    chunk_words: int = 180,
    overlap: int = 40
) -> List[Dict]:
    """Fixed-size word window chunking with overlap.

    Why: consistent chunk sizes improve embedding stability, especially on OCR/noisy text.
    Trade-off: may cut across sentence boundaries.
    """
    words = text.split()
    chunks: List[Dict] = []

    step = max(1, chunk_words - overlap)
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i + chunk_words]).strip()
        if len(chunk) < 120:
            continue
        chunks.append({"source": source_name, "page": page_num + 1, "text": chunk})
    return chunks


def chunk_sentence_based(
    text: str,
    source_name: str,
    page_num: int,
    max_words: int = 220
) -> List[Dict]:
    """Sentence-preserving chunking.

    Why: keeps sentences intact → better readability and cleaner citations.
    Great for: summaries / contributions / overview questions.
    Trade-off: chunk sizes vary; long paragraphs can create larger chunks.
    """
    sentences = [s.strip() for s in safe_sentence_split(text) if len(s.strip()) > 20]
    chunks: List[Dict] = []
    current: List[str] = []
    current_words = 0

    for sent in sentences:
        w = len(sent.split())

        # Hard cap extremely long sentences
        if w > max_words:
            sent = " ".join(sent.split()[:max_words])
            w = len(sent.split())

        if current_words + w > max_words and current:
            chunks.append({"source": source_name, "page": page_num + 1, "text": " ".join(current)})
            current = []
            current_words = 0

        current.append(sent)
        current_words += w

    if current:
        chunks.append({"source": source_name, "page": page_num + 1, "text": " ".join(current)})

    # Filter tiny chunks
    return [c for c in chunks if len(c["text"]) >= 120]


def chunk_text(
    text: str,
    source_name: str,
    page_num: int,
    mode: str = "auto"
) -> List[Dict]:
    """Unified chunking interface.

    mode:
      - "word"      → word-window chunking
      - "sentence"  → sentence/paragraph-preserving
      - "auto"      → choose based on text length (long/OCR-like → word)
    """
    mode = (mode or "auto").lower().strip()
    if mode == "word":
        return chunk_word_window(text, source_name, page_num)
    if mode == "sentence":
        return chunk_sentence_based(text, source_name, page_num)

    # AUTO heuristic: long pages tend to work better with word windows (especially OCR output)
    word_count = len((text or "").split())
    if word_count > 900:
        return chunk_word_window(text, source_name, page_num)
    return chunk_sentence_based(text, source_name, page_num)


# EMBEDDING + INDEXING (Chroma)

#### Session-safe Chroma store (unique temp dir per reset)

In [11]:
SESSION_DIR = None
chroma_client = None
collection = None

def create_vector_store() -> chromadb.api.models.Collection.Collection:
    """
    Embedding/Index stage setup:
    Uses a unique temporary directory per session to avoid conflicts/read-only issues.
    """
    global SESSION_DIR, chroma_client, collection

    if SESSION_DIR and os.path.exists(SESSION_DIR):
        shutil.rmtree(SESSION_DIR, ignore_errors=True)

    SESSION_DIR = tempfile.mkdtemp(prefix="chroma_session_")
    chroma_client = chromadb.PersistentClient(path=SESSION_DIR)

    emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key,
        model_name="text-embedding-3-small"
    )

    collection = chroma_client.get_or_create_collection(
        name="pdf_rag_collection",
        embedding_function=emb_fn
    )
    return collection

create_vector_store()
print("✅ Chroma initialized:", SESSION_DIR)

✅ Chroma initialized: /tmp/chroma_session_c8935s7j


#### Indexing: store chunks + metadata in Chroma

In [12]:
def index_document(file_path: str, source_name: str, chunk_mode: str = "auto") -> str:
    """Embedding/Indexing stage:
    - Extract pages with a per-page cascade (fitz → pdfplumber → OCR)
    - Chunk pages using selectable strategies (word-window, sentence-preserving, auto)
    - Add to Chroma with collision-safe UUID IDs

    Adds a reviewer-friendly debug line to the status:
      "Chunking used: <selected> → <resolved modes> | Chunks: N"
    where <resolved modes> is:
      - "word" or "sentence" if a fixed mode is chosen
      - "word/sentence" (or whichever occurred) if chunk_mode="auto"

    Returns a human-readable status string for the UI.
    """
    pages, stats = extract_pages_with_cascade(file_path)

    all_chunks: List[Dict] = []
    resolved_modes_used = set()
    chunk_counts_by_mode = {"word": 0, "sentence": 0}

    for page_num, page_text in enumerate(pages):
        if not page_text or not page_text.strip():
            continue

        # Determine which strategy was effectively used (for reporting)
        if chunk_mode == "auto":
            # Mirror the heuristic used in chunk_text()
            resolved = "word" if len(page_text.split()) > 900 else "sentence"
        else:
            resolved = chunk_mode

        resolved_modes_used.add(resolved)

        page_chunks = chunk_text(page_text, source_name, page_num, mode=chunk_mode)
        all_chunks.extend(page_chunks)

        if resolved in chunk_counts_by_mode:
            chunk_counts_by_mode[resolved] += len(page_chunks)

    if not all_chunks:
        # Clearer, actionable message
        if not stats.get("pytesseract_available", False):
            return (
                "❌ No readable text extracted. This PDF may be image-based. "
                "Optional OCR is disabled (pytesseract/tesseract not available).\n\n"
                f"Extraction stats: {stats}"
            )
        return (
            "❌ No readable text extracted even after OCR fallback.\n\n"
            f"Extraction stats: {stats}"
        )

    ids = [uuid.uuid4().hex for _ in all_chunks]  # avoids collisions on re-index
    collection.add(
        documents=[c["text"] for c in all_chunks],
        metadatas=[{"source": c["source"], "page": c["page"], "text": c["text"]} for c in all_chunks],
        ids=ids,
    )

    resolved_str = "/".join(sorted(resolved_modes_used)) if resolved_modes_used else chunk_mode
    chunking_debug = f"Chunking used: {chunk_mode} → {resolved_str} | Chunks: {len(all_chunks)}"
    if chunk_mode == "auto":
        chunking_debug += f" (word: {chunk_counts_by_mode['word']}, sentence: {chunk_counts_by_mode['sentence']})"

    return (
        f"✅ Document Indexed Successfully\n"
        f"{chunking_debug}\n"
        f"Pages: {stats['pages_total']} | fitz: {stats['fitz_pages_with_text']} | "
        f"pdfplumber: {stats['plumber_pages_with_text']} | OCR: {stats['ocr_pages_with_text']} | "
        f"empty: {stats['empty_pages']}"
    )


# RETRIEVAL 

#### Retrieval: query Chroma (fallback if empty)

In [13]:
def retrieve(query: str, k: int = 6) -> Tuple[List[str], List[Dict]]:
    """
    Retrieval stage:
    Query Chroma and return (docs, metas).
    """
    query = (query or "").strip()
    if not query:
        return [], []

    results = collection.query(query_texts=[query], n_results=k)
    docs = results.get("documents", [[]])[0]
    metas = results.get("metadatas", [[]])[0]
    return docs, metas

# GENERATION (LLM) + CITATIONS
#### Generate grounded answer; suppress citations if “no relevant info”

In [ ]:
def is_summary_intent(query: str) -> bool:
    """Heuristic intent detector for summary-style questions.

    Why:
    - Summary/contribution questions need broader context than factual Q&A.
    - We can automatically increase Top-K and switch to a summary prompt.
    """
    q = (query or "").lower()
    triggers = [
        "summarize", "summary", "overview", "main contribution", "main contributions",
        "key contributions", "what is this paper about", "abstract", "tl;dr", "tldr"
    ]
    return any(t in q for t in triggers)


In [14]:
def generate_answer(query: str, docs: List[str], metas: List[Dict], summary_mode: bool = False) -> Tuple[str, List[Dict]]:
    """Generation stage (direct OpenAI; no LangChain).

    - If `summary_mode` is True, produce a structured paper-style summary.
    - If no docs are retrieved, return a clear 'no relevant information' response with no citations.
    """
    if not docs:
        return "The provided context does not contain relevant information.", []

    per_chunk_cap = 900 if not summary_mode else 1100

    context_blocks = []
    for d, m in zip(docs, metas):
        context_blocks.append(f"[{m['source']} | Page {m['page']}] {d[:per_chunk_cap]}")
    context = "\n\n".join(context_blocks)

    if summary_mode:
        prompt = f"""
You are a research assistant.

Task: Produce a concise but informative summary of the paper using ONLY the provided context.

Output format:
- **Problem / Motivation** (1-2 bullets)
- **Approach / Method** (2-4 bullets)
- **Main Contributions** (3-6 bullets)
- **Key Results / Claims** (1-3 bullets)
- **Limitations / Open Questions** (1-3 bullets)

Rules:
- Use ONLY the context.
- If something is missing, explicitly say it is missing.
- Add citations at the end of each bullet in the format [Source, Page X].

Context:
{context}
"""
    else:
        prompt = f"""
You are a research assistant.

Rules:
- Answer ONLY using the provided context.
- If the answer is not in the context, say exactly:
  "The provided context does not contain relevant information."
- Keep the answer concise (5-8 sentences max).
- Add citations where appropriate in the format [Source, Page X].
- Do not invent citations.

Context:
{context}

Question:
{query}
"""

    try:
        resp = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )
        answer = resp.choices[0].message.content.strip()

        if "does not contain relevant information" in answer.lower():
            return answer, []

        return answer, metas

    except Exception as e:
        return f"❌ LLM generation failed: {str(e)}", []


#### Citation formatting (300 chars, no mid-word cut)

In [15]:
def format_sources(metas: List[Dict], max_chars: int = 300) -> str:
    """
    Formats citations for UI:
    - includes source + page + short snippet (no mid-word cuts)
    - dedupes repeated items
    """
    if not metas:
        return "No citations."

    lines = ["### 📚 Sources"]
    seen = set()

    for m in metas:
        raw = (m.get("text") or "").replace("\n", " ").strip()
        key = (m.get("source"), m.get("page"), raw[:120])
        if key in seen:
            continue
        seen.add(key)

        if len(raw) > max_chars:
            cut = raw.rfind(" ", 0, max_chars)
            cut = cut if cut != -1 else max_chars
            raw = raw[:cut].rstrip() + "..."

        lines.append(f'- **{m.get("source","Unknown")}** (Page {m.get("page","?")}): "{raw}"')

    return "\n".join(lines)

# UI + Reset
#### Reset: clears DB + clears UI fields safely

In [16]:
def clear_all():
    """
    Reset button:
    - creates a fresh Chroma session
    - returns correct types for Gradio components
    """
    create_vector_store()
    return (
        "",     # answer_output
        "",     # sources_output
        "",     # status_box
        None,   # file_input
        "",     # url_input
        ""      # query_input
    )

#### Debug formatting helper

In [17]:
def format_debug_retrieval(docs: List[str], metas: List[Dict], max_chars: int = 450) -> str:
    """
    Debug panel:
    Shows what the retriever returned (top-k) so reviewers can inspect RAG behavior.
    """
    if not docs:
        return "No retrieved chunks (empty retrieval result)."

    lines = ["### 🧪 Retrieval Debug (Top-K Chunks)"]
    for i, (d, m) in enumerate(zip(docs, metas), start=1):
        raw = (d or "").replace("\n", " ").strip()
        if len(raw) > max_chars:
            cut = raw.rfind(" ", 0, max_chars)
            cut = cut if cut != -1 else max_chars
            raw = raw[:cut].rstrip() + "..."

        lines.append(
            f"**{i}. {m.get('source','Unknown')} — Page {m.get('page','?')}**\n\n"
            f"> {raw}\n"
        )
    return "\n\n".join(lines)


#### Process handler (supports debug toggle)

In [20]:
def process_input(file, url, query, k, debug_mode, chunk_mode):
    """Gradio handler.

    Behaviors:
    - If a file/URL is provided, we reset the session store and re-index the document.
    - We auto-detect summary-style questions and widen retrieval (higher Top-K) + switch prompt.
    - If retrieval returns no hits, we return a clear answer with *no citations* and update status.
    """
    try:
        if not query or not query.strip():
            return "Error: Please type your question here.", "", "❌ Missing question.", ""

        status = "Using existing indexed document."

        # New doc provided -> reset + index
        if file or (url and url.strip()):
            create_vector_store()
            local_path, source_name = ingest_pdf(file=file, url=url.strip() if url else None)
            status = index_document(local_path, source_name, chunk_mode=chunk_mode)

        q = query.strip()
        summary_mode = is_summary_intent(q)

        # Broader retrieval for summaries/contributions
        eff_k = max(int(k), 10) if summary_mode else int(k)

        docs, metas = retrieve(q, k=eff_k)

        debug_text = format_debug_retrieval(docs, metas) if debug_mode else ""

        if not docs:
            status = status + "\n⚠️ Retrieval returned 0 relevant chunks. Try increasing Top-K or re-indexing."
            return "The provided context does not contain relevant information.", "No citations.", status, debug_text

        answer, used = generate_answer(q, docs, metas, summary_mode=summary_mode)
        citations = format_sources(used, max_chars=300)

        return answer, citations, status, debug_text

    except Exception as e:
        return f"Error: {str(e)}", "", "❌ Failed.", ""


# Gradio UI

In [25]:
with gr.Blocks(theme=gr.themes.Soft(), title="PDF RAG (No LangChain)") as demo:
    gr.Markdown(
        "# 📄 PDF RAG Assistant (No LangChain)\n"
        "Upload a PDF or paste a PDF URL. Ask questions and get answers with **source + page** citations."
    )

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📥 Document")
            file_input = gr.File(label="Upload PDF", file_types=[".pdf"])
            url_input = gr.Textbox(label="Or PDF URL", placeholder="https://arxiv.org/pdf/1706.03762.pdf")

            gr.Markdown("### ⚙️ Retrieval")
            k_slider = gr.Slider(2, 10, value=6, step=1, label="Top-K chunks")
            debug_mode = gr.Checkbox(value=False, label="Show retrieval debug")

            chunk_mode = gr.Dropdown(
                choices=["auto", "sentence", "word"],
                value="auto",
                label="Chunking strategy",
                info="auto=best default • sentence=best for summaries • word=best for OCR/noisy text"
            )

            status_box = gr.Textbox(label="Status", value="Ready.", interactive=False)

            clear_btn = gr.Button("Clear / Reset session", variant="primary")

        with gr.Column():
            gr.Markdown("### 💬 Ask")
            query_input = gr.Textbox(
                label="Type your question here",
                placeholder="e.g., What is self-attention and why is it useful?",
                lines=2
            )
            ask_btn = gr.Button("Ask", variant="primary")

            gr.Markdown("### ✅ Answer")
            answer_output = gr.Markdown()

            gr.Markdown("### 📚 Citations")
            sources_output = gr.Markdown()

            with gr.Accordion("🧪 Retrieval Debug (Top-K chunks)", open=False):
                debug_output = gr.Markdown(value="(Enable 'Show retrieval debug' to display retrieved chunks.)")

    ask_btn.click(
        process_input,
        inputs=[file_input, url_input, query_input, k_slider, debug_mode, chunk_mode],
        outputs=[answer_output, sources_output, status_box, debug_output]
    )

    clear_btn.click(
        clear_all,
        outputs=[answer_output, sources_output, status_box, file_input, url_input, query_input]
    ).then(
        lambda: "(Enable 'Show retrieval debug' to display retrieved chunks.)",
        outputs=debug_output
    )

demo.launch(share=True)


/tmp/ipykernel_21346/4031861923.py:1: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="PDF RAG (No LangChain)") as demo:


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://22b606a89c1c569df5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
